##**Script for Calculating Topographic Predictors and Extracting Zonal Statistics**

### **Setup Script**

**Import common python libs**

In [1]:
import os
import sys
import datetime
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
import math
import csv
import numpy as np                  # to create a sequence for plotting
from scipy.spatial import distance  # for Jensen-Shannon
import matplotlib.pyplot as plt     # for plotting histograms
import pandas as pd                 # for creating histogram dataframe to export to GDrive
from google.colab import drive      # for exporting from distributed machine to GDrive
from google.colab import files

**Install, import, & authenticate earthengine python API**

In [2]:
##reference: https:#developers.google.com/earth-engine/python_install_manual

!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 2.6MB 7.9MB/s 


In [3]:
##@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=uNwZz1paMgQU0CktI3o_1MRWdLQUApTGtTuPE_yCvKA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4gFUvNJ4POcovU1s9kAHlmHpDDQfIk_EJF88ufqld9L6zMpWNHoFsiI

Successfully saved authorization token.
{'t

### **Begin calculating Biophysical Predictors**

**Import Geometries for Great Basin and AIM plots (with buffers)**

In [4]:
# CHANGE GEOMETRY DEPENDING ON WHAT YOU'RE EXPORTING
GBbb = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/SaddleDraw').first().geometry()
# GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/GBbounds')
# GBbb = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/GBboundingbox')
# GBbb = ee.Feature(GBbounds.first().geometry().bounds())

AIMplots = ee.FeatureCollection('users/ericjensen41_default/Thesis/Plots/Allplots')
# fire_pts = ee.FeatureCollection('users/ericjensen41_default/Thesis/Chapter2/Fire_points')
fire_pts = ee.FeatureCollection('users/ericjensen41_default/Thesis/Chapter2/Fire_pointsAll')

# Buffer function to apply to plots
def buffer100(f):
  return f.buffer(100)

AIMplots = AIMplots.map(buffer100)

print(fire_pts.first().getInfo())
# print(GBbounds.getInfo())
print(GBbb.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-115.2700452913153, 42.285401120128064]}, 'id': '00000000000000000fac', 'properties': {'Fire_Name': 'TUANA COMPLEX', 'ID': 'TUA_1', 'Year': 1995}}
{'type': 'Polygon', 'coordinates': [[[-118.36875932166807, 43.30935340433893], [-118.36868801703291, 43.308898560233075], [-118.36838921382908, 43.3084839009838], [-118.3678185214122, 43.308229740833305], [-118.36722542693398, 43.30839025451931], [-118.36676169883219, 43.308706869893115], [-118.36641382740625, 43.30911707828469], [-118.36616863840231, 43.309406909706325], [-118.36606601175832, 43.30952734700599], [-118.36560229660589, 43.30984388290856], [-118.36500924120614, 43.31000444598895], [-118.36441612145936, 43.31016494834059], [-118.36384092333455, 43.310378985285546], [-118.3632478458208, 43.31054399265663], [-118.36265487419446, 43.310704517686744], [-118.36207957270571, 43.31091857140857], [-118.36152220492724, 43.311190533375246], [-118.36096480042217, 43.3114625

**Import input datasets from Conservation Science partners for calculating values**

In [5]:
# US NED CHILI (Continuous Heat-Insolation Load Index)
csp_chili = ee.Image('CSP/ERGo/1_0/US/CHILI').rename('csp_chili').resample('bicubic')
# US NED mTPI (Multi-Scale Topographic Position Index)
csp_tpi = ee.Image('CSP/ERGo/1_0/US/mTPI').rename('csp_tpi').resample('bicubic')
# US NED Topographic Diversity
csp_topodiv = ee.Image('CSP/ERGo/1_0/US/topoDiversity').rename('csp_topodiv').resample('bicubic')
# US NED Landforms
csp_landforms = ee.Image('CSP/ERGo/1_0/US/landforms').rename('csp_landforms').resample('bicubic')
# US NED Physiographic Diversity
csp_physd = ee.Image('CSP/ERGo/1_0/US/physioDiversity').rename('csp_physd').resample('bicubic')

**Calculate additional biophysical predictors**

In [6]:
# Latitude and longitude
latlong = ee.Image.pixelLonLat(); 

# Topography
elevation = ee.Image('USGS/NED').rename('elev').resample('bicubic')
aspect = ee.Terrain.aspect(elevation).multiply(ee.Number(math.pi).divide(ee.Number(180))).rename('aspect') # linear aspect
northness = aspect.cos().rename('northness')
eastness = aspect.sin().rename('eastness')
slope = ee.Terrain.slope(elevation)

# Stage 1975: slope% * cos or sin of aspect
slope_pct = slope.expression("tan(b(0) * pi/180)", {"pi": ee.Number(math.pi)}).rename("slope_pct")
slope_pct = slope_pct.where(slope_pct.gt(1), 1.01)

slope_east = slope_pct.multiply(eastness).rename('slope_east')
slope_north = slope_pct.multiply(northness).rename('slope_north')

# TRASP - Roberts and Cooper 1989
trasp = aspect.expression("(1-(cos(b(0)-d))) / 2", {"d":(ee.Number(30).multiply(ee.Number(math.pi).divide(ee.Number(180))))}).rename("trasp")

# Topographic position index
tpi90 = elevation.subtract(elevation.focal_mean(radius=90, units='meters')).rename('tpi90')
tpi990 = elevation.subtract(elevation.focal_mean(radius=990, units='meters')).rename('tpi990')

**Create single multi-band image with all of the topographic bands of interest**

In [7]:
# add topographic bands to the image for calculating means
topo_mean = ee.Image.cat(latlong, elevation, aspect, northness, eastness, slope_pct, slope_east, slope_north, trasp, tpi90, tpi990, csp_chili, csp_tpi, csp_topodiv, csp_physd)
# add topographic bands to the image for calculating means
topo_mode = ee.Image.cat(csp_landforms)

# Reference table to landforms dataset
# 11	141414	Peak/ridge (warm)
# 12	383838	Peak/ridge
# 13	808080	Peak/ridge (cool)
# 14	EBEB8F	Mountain/divide
# 15	F7D311	Cliff
# 21	AA0000	Upper slope (warm)
# 22	D89382	Upper slope
# 23	DDC9C9	Upper slope (cool)
# 24	DCCDCE	Upper slope (flat)
# 31	1C6330	Lower slope (warm)
# 32	68AA63	Lower slope
# 33	B5C98E	Lower slope (cool)
# 34	E1F0E5	Lower slope (flat)
# 41	a975ba	Valley
# 42	6f198c	Valley (narrow)

topo_mean_bands = topo_mean.bandNames().getInfo()
print(topo_mean_bands)

topo_mode_bands = topo_mode.bandNames().getInfo()
print(topo_mode_bands)

['longitude', 'latitude', 'elev', 'aspect', 'northness', 'eastness', 'slope_pct', 'slope_east', 'slope_north', 'trasp', 'tpi90', 'tpi990', 'csp_chili', 'csp_tpi', 'csp_topodiv', 'csp_physd']
['csp_landforms']


### **Calculate zonal statistics and export**

In [ ]:
def mean_export(image):
  mean_FC = image.reduceRegions(reducer = ee.Reducer.mean(), collection = fire_pts, scale = 30)
  mean_exp_task = ee.batch.Export.table.toDrive(collection = mean_FC,
                                            description = 'Biophysical_Predictors_Mean', 
                                            fileNamePrefix = 'Biophysical_Predictors_Mean',
                                            fileFormat = 'CSV')
  mean_exp_task.start()

In [ ]:
def mode_export(image):
  mode_FC = image.reduceRegions(reducer = ee.Reducer.mode(), collection = fire_pts, scale = 30)
  mode_exp_task = ee.batch.Export.table.toDrive(collection = mode_FC,
                                            description = 'Biophysical_Predictors_Mode', 
                                            fileNamePrefix = 'Biophysical_Predictors_Mode',
                                            fileFormat = 'CSV')
  mode_exp_task.start()

In [ ]:
mean_export(topo_mean)

In [ ]:
mode_export(topo_mode)

### **Export top predictor images for species richness**

**Create focal grids for each image to match scale of calculations in the model**

In [ ]:
# Select bands for the model export
topo_I = topo_mean.select(['slope_pct', 'csp_topodiv', 'elev'])

# Reproject and resample image by nearest neighbors to match Landsat
LS_ref = ee.Image('users/zackrwerner/landsat_harm_reference')
LS_proj = LS_ref.projection() # get projection of landsat harmonized image
topo_I_resample = topo_I.resample().reproject(LS_proj) #reproject defaults to nn 

# Calculate focal means at 100 meter scale using reduceNeighborhood
topo_I_resample_focal = topo_I_resample.reduceNeighborhood(reducer = ee.Reducer.mean(), kernel = ee.Kernel.circle(100, 'meters')).rename(['p.slope_pct', 'p.csp_topodiv', 'p.elev'])

In [ ]:
# Get list of bandnames to export over
namelist = topo_I_resample_focal.bandNames().getInfo();
print(namelist,'namelist');

for i in namelist:
  # Get image
  img = topo_I_resample_focal.select(i).clip(GBbounds.geometry());
  
  # Export results to drive
  topo_exp_task = ee.batch.Export.image.toDrive(
      image =  img,
      description = i,
      scale = 30,
      maxPixels = 1e13,
      region = GBbounds.geometry()) 
  
  topo_exp_task.start()


['slope_pct', 'csp_topodiv', 'elev', 'csp_physd', 'csp_tpi', 'tpi90'] namelist


### **Export top predictor images for recovery modeling**

In [8]:
# Select bands for the model export
topo_I = topo_mean.select(['csp_chili','slope_north'])

# Reproject and resample image by nearest neighbors to match Landsat
LS_ref = ee.Image('users/zackrwerner/landsat_harm_reference')
LS_proj = LS_ref.projection() # get projection of landsat harmonized image
topo_I_resample = topo_I.resample().reproject(LS_proj) #reproject defaults to nn 

In [9]:
# Get list of bandnames to export over
namelist = topo_I_resample.bandNames().getInfo();
print(namelist,'namelist');

for i in namelist:
  # Get image
  img = topo_I_resample.select(i).clip(GBbb);
  
  # Export results to drive
  topo_exp_task = ee.batch.Export.image.toDrive(
      image =  img,
      description = i + '_pst',
      scale = 30,
      maxPixels = 1e13,
      region = GBbb) 
  
  topo_exp_task.start()

['csp_chili', 'slope_north'] namelist
